## MNIST Handwritten Digit Recognition

[MNIST](http://yann.lecun.com/exdb/mnist) is often considered the [hello world](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program) of machine learning. It contains 70,000 images of handwritten digits suitable for image classification. Typically, 60,000 of these images are used for training and 10,000 are set aside for testing. Each digit is a black and white 28 x 28 pixel image along with a label, so the full dataset can be used for supervised training as well as testing unsupervised algorithms.

Over the years, MNIST has been used to measure the accuracy of a number of different algorithms. In this example, we will train a standard three layer neural network. Even this simple model can achieve 98% accuracy, rivaling human performance.

This example requires Python 3, Keras, Tensorflow, matplotlib, and numpy. You can use either Tensorflow-CPU or Tensorflow-GPU, of course Tensorflow-GPU will train significantly faster.

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from matplotlib import pyplot as plt
import numpy as np
import random

These are some configuration parameters and hyperparameters.

In [ ]:
# The number of training samples per batch. 128 is a reasonable number.
batch_size = 128

# Our data set contains 10 digits, so the number of classes is 10
num_classes = 10

# epochs is the number of times the model is trained with the data set, more can be better, up to a point
epochs = 5

# dropout is a common regularization hyperperameter. It helps to avoid overfitting or memorizing the input.
dropout = 0.5

### Load data

Keras has a builting function for loading MNIST data while splitting it into train and test sets. x_train and x_test are arrays if train and test input images respectively. Images are represented as a 28 x 28 matrix of pixel values. y_train and y_test are train and test labels respectively.

The image matrices are reshaped into vectors suitable for training. Pixel values are normalized into values ranging from 0.0 - 1.0.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Save original test images for display later
orig_test = x_test

# Reshape and normalize data
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Define Model

This is the neural network model itself built using predefined Keras classes. This is a simple three layer model using dense, fully connected layers. Still, it achieves very good accuracy which demonstrates the predictive power of properly trained neural networks. The input shape to the model is an image represented as a 784 element (28 x 28) vector.

The first two layers use a REctified Linear Unit activation function, which does not suffer from the "vanishing gradient" problem that plagued traditional sigmoid activation functions in the past. The output layer has 10 neurons corresponding to one of 10 possible digit classes. The neuron with the largest output is the most likely digit. This is known as one-hot encoding, a concept borrowed from digital logic.

The best image recognition models use Convolutional Neural Networks (CNNs), which are like two dimensional weighted filters. The inpiration for CNNs comes from the visual cortex of animal brains. Models with over 100 CNN, pooling, padding, and other layers have been successfully trained to recognize complex objects as well as multiple objects in the one image.

In [ ]:
# Create three layer model, two hidden layers, one output layer
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(dropout))
model.add(Dense(512, activation='relu'))
model.add(Dropout(dropout))
# 10 output units, one for each digit (one hot encoding)
model.add(Dense(10, activation='softmax'))

model.summary()

### Train Model

This is more or less boilerplate Keras code to train the model given the training data set. Keras can also evaluate the trained model given the test data set. Matplotlib is used to plot train and test accuracy as a function of the epoch number.

Ideally, the accuracy with test and train data should be about the same. Low training accuracy means the model exhibits high bias. High bias could be caused by a weak model, not enough training examples, or too few training epochs. High train accuracy and low test accuracy is a case of high variance. This means the model is more or less "memorizing" the expected output and not generalizing well. High variance can be reduced through *regularization*.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

# Train model for a given number of epochs
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# Evaluate model against test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### Make Predictions

Use the trained model to make predictions on data that it was not trained on.
Notice that prediction takes much less time compared with training.

Keras can make predictions for a set of test data, rather than just one sample. In this case, the model is used to make predictions for all of the test data in one statement. Keras/Tensorflow will parallelize this operation as much as possible when using a GPU.

The predicted output is a one-hot vector. argmax() is used to translate the output vector to a scalar value by choosing the index of the vector with the highest value.

In [ ]:
predictions = model.predict(x_test).argmax(1)
y_test_values = y_test.argmax(1)

The following snippet chooses a random test image, prints the model predicted value, expected value (label), and the original image.

In [ ]:
index = random.randrange(10000)
plt.imshow(orig_test[index], cmap='gray')
print('Prediction: %d, label: %d' % (predictions[index], y_test_values[index]))

### Error Analysis

The next snippet looks for model predictions that specifically do not match the expected value. In some cases, even humans might disagree on what this image is.

In [ ]:

for i in range(10000):
    index = random.randrange(10000)
    if predictions[index] != y_test_values[index]:
        break
plt.imshow(orig_test[index], cmap='gray')
print('Prediction: %d, label: %d' % (predictions[index], y_test_values[index]))

The following snippet calculates the error rate by comparing predicted values with expected values. This is similar to what Keras does when evaluating a model.

In [ ]:
total = 0.0
misclassified = 0.0
for i in range(10000):
    total += 1
    if predictions[i] != y_test_values[i]:
        misclassified += 1
print('Error rate: %.2f %%' % (100.0*misclassified/total))